# 基于qlib的量化交易系统快速入门本笔记本展示了如何使用基于qlib的量化交易系统进行数据分析和策略回测。

## 1. 环境准备首先导入必要的库并设置路径。

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# 添加项目根目录到系统路径
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# 设置绘图样式
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')

# 显示设置
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

## 2. 导入项目模块导入我们构建的量化系统模块。

In [ ]:
# 导入项目模块
from data.fetcher import DataFetcher
from data.processor import DataProcessor
from strategies.simple_strategies import MomentumStrategy, MeanReversionStrategy, MovingAverageCrossStrategy, RSIStrategy
from backtest.backtest_engine import BacktestEngine
from backtest.analyzer import BacktestAnalyzer
from results.visualizer import Visualizer

## 3. 获取股票数据使用DataFetcher获取股票数据。

In [ ]:
# 创建数据目录
os.makedirs('../data/raw', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)

# 设置股票和时间范围
symbols = ['000001.SZ', '600000.SH', '600519.SH', '000858.SZ', '601318.SH']
start_date = '2018-01-01'
end_date = '2021-12-31'

# 尝试加载本地数据，如果不存在则从网络获取
data_path = '../data/raw/stock_data.csv'

try:
    # 尝试使用本地数据
    data = pd.read_csv(data_path)
    print(f"已加载本地数据，共 {len(data)} 条记录")
except FileNotFoundError:
    # 如果本地数据不存在，则从网络获取
    print("本地数据不存在，从网络获取数据...")
    fetcher = DataFetcher()
    data = fetcher.batch_fetch_stock_data(symbols, start_date, end_date, source='yahoo')
    data.to_csv(data_path, index=False)
    print(f"数据获取完成，共 {len(data)} 条记录")

查看数据的前几行，了解数据结构。

In [ ]:
# 查看数据结构
data.head()

检查数据的基本统计信息。

In [ ]:
# 查看数据统计信息
data.describe()

## 4. 数据处理使用DataProcessor处理股票数据，计算技术指标。

In [ ]:
# 处理数据
processor = DataProcessor()
processed_data = processor.process_stock_data(data)

# 保存处理后的数据
processed_data.to_csv('../data/processed/processed_data.csv', index=False)
print(f"数据处理完成，共 {len(processed_data)} 条记录")

查看处理后的数据，包含了计算的技术指标。

In [ ]:
# 查看处理后的数据
processed_data.head()

## 5. 数据可视化绘制股票价格走势图。

In [ ]:
# 选择一只股票进行可视化
stock_id = '000001.SZ'
stock_data = processed_data[processed_data['stock_id'] == stock_id].copy()

# 将日期列转换为日期类型
stock_data['date'] = pd.to_datetime(stock_data['date'])

# 绘制价格走势图
plt.figure(figsize=(12, 6))
plt.plot(stock_data['date'], stock_data['close'], label='收盘价')
plt.title(f'{stock_id} 价格走势')
plt.xlabel('日期')
plt.ylabel('价格')
plt.grid(True)
plt.legend()
plt.show()

绘制技术指标。

In [ ]:
# 绘制移动平均线
plt.figure(figsize=(12, 6))
plt.plot(stock_data['date'], stock_data['close'], label='收盘价')
plt.plot(stock_data['date'], stock_data['ma5'], label='5日均线')
plt.plot(stock_data['date'], stock_data['ma20'], label='20日均线')
plt.title(f'{stock_id} 移动平均线')
plt.xlabel('日期')
plt.ylabel('价格')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# 绘制RSI指标
plt.figure(figsize=(12, 6))
plt.plot(stock_data['date'], stock_data['rsi_14'], label='RSI(14)')
plt.axhline(y=70, color='r', linestyle='--', label='超买线(70)')
plt.axhline(y=30, color='g', linestyle='--', label='超卖线(30)')
plt.title(f'{stock_id} RSI指标')
plt.xlabel('日期')
plt.ylabel('RSI值')
plt.grid(True)
plt.legend()
plt.show()

## 6. 策略回测创建回测引擎和策略，运行回测。

In [ ]:
# 创建回测引擎
backtest_config = {
    'start_date': '2019-01-01',  # 回测开始日期
    'end_date': '2021-12-31',  # 回测结束日期
    'benchmark': '000001.SZ',  # 基准指数
    'account': 1000000,  # 初始资金
    'commission_rate': 0.0003,  # 手续费率
    'slippage_rate': 0.0001,  # 滑点率
    'trade_frequency': 'day',  # 交易频率
    'verbose': True  # 是否打印详细信息
}
backtest_engine = BacktestEngine(backtest_config)

In [ ]:
# 创建策略
# 1. 移动平均线交叉策略
ma_cross_strategy = MovingAverageCrossStrategy(short_window=5, long_window=20)

# 2. RSI策略
rsi_strategy = RSIStrategy(rsi_period=14, overbought=70, oversold=30)

# 3. 动量策略
momentum_strategy = MomentumStrategy(lookback_period=20, holding_period=5)

# 4. 均值回归策略
mean_reversion_strategy = MeanReversionStrategy(lookback_period=10, threshold=1.5)

In [ ]:
# 选择一个策略进行回测
strategy = ma_cross_strategy  # 可以替换为其他策略

# 运行回测
results = backtest_engine.run_backtest(strategy, processed_data)

# 保存回测结果
os.makedirs('../results', exist_ok=True)
backtest_engine.save_results('../results/backtest_results.json')

## 7. 回测结果分析使用BacktestAnalyzer分析回测结果。

In [ ]:
# 创建分析器
analyzer = BacktestAnalyzer(results)

# 获取回测摘要
summary = analyzer.get_summary()
print("回测摘要:")
print(summary)

In [ ]:
# 绘制权益曲线
analyzer.plot_equity_curve()

In [ ]:
# 绘制回撤曲线
analyzer.plot_drawdown()

In [ ]:
# 绘制月度收益热力图
analyzer.plot_monthly_returns()

## 8. 比较不同策略运行多个策略并比较它们的性能。

In [ ]:
# 创建策略列表
strategies = [
    MovingAverageCrossStrategy(short_window=5, long_window=20),
    RSIStrategy(rsi_period=14, overbought=70, oversold=30),
    MomentumStrategy(lookback_period=20, holding_period=5),
    MeanReversionStrategy(lookback_period=10, threshold=1.5)
]

# 存储结果
strategy_results = {}
strategy_metrics = {}

# 运行每个策略的回测
for strategy in strategies:
    strategy_name = strategy.__class__.__name__
    print(f"运行 {strategy_name} 回测...")
    
    backtest_engine = BacktestEngine(backtest_config)
    results = backtest_engine.run_backtest(strategy, processed_data)
    
    # 保存结果
    strategy_results[strategy_name] = results
    
    # 获取关键指标
    metrics = {
        'annual_return': results['annual_return'],
        'sharpe_ratio': results['sharpe_ratio'],
        'max_drawdown': results['max_drawdown'],
        'win_rate': results['win_rate']
    }
    strategy_metrics[strategy_name] = metrics

In [ ]:
# 比较策略性能
metrics_df = pd.DataFrame(strategy_metrics).T
print("策略性能比较:")
print(metrics_df)

In [ ]:
# 使用Visualizer绘制策略对比图
visualizer = Visualizer()

# 绘制权益曲线对比
plt.figure(figsize=(12, 6))

# 添加基准曲线
benchmark_values = strategy_results[strategies[0].__class__.__name__]['benchmark_value']
dates = strategy_results[strategies[0].__class__.__name__]['dates']
plt.plot(dates, benchmark_values, label='基准', linestyle='--')

# 添加每个策略的权益曲线
for strategy in strategies:
    strategy_name = strategy.__class__.__name__
    portfolio_values = strategy_results[strategy_name]['portfolio_value']
    plt.plot(dates, portfolio_values, label=strategy_name)

plt.title('策略权益曲线对比')
plt.xlabel('日期')
plt.ylabel('价值')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 绘制指标对比条形图
visualizer.plot_metrics_comparison(strategy_metrics, metrics=['annual_return', 'sharpe_ratio', 'max_drawdown', 'win_rate'])

## 9. 生成回测报告生成完整的回测报告。

In [ ]:
# 选择一个策略的结果生成报告
best_strategy_name = metrics_df['sharpe_ratio'].idxmax()
print(f"生成 {best_strategy_name} 的回测报告")

analyzer = BacktestAnalyzer(strategy_results[best_strategy_name])
analyzer.generate_report(output_dir=f'../results/{best_strategy_name}')

## 10. 总结在本笔记本中，我们展示了如何使用基于qlib的量化交易系统进行数据获取、处理、策略回测和结果分析。通过比较不同策略的性能，我们可以选择最适合的策略进行实际交易。请注意，回测结果仅供参考，实际交易中还需考虑更多因素，如流动性、市场冲击等。